# Hugging face dataset

In [ ]:
!tensorboard --logdir=/home/akinwilson/Code/pytorch/output/model/DeepSpeech/2022_09_24_12.18.00_PM/tf_board

In [2]:
import base64
import numpy as np 
import os 

print(os.environ['DATA_ROOT'])

/media/akinwilson/Samsung_T5/data/audio/keyword-spotting


In [ ]:
# import scipy.io  as io
path = "/home/akinwilson/Code/pytorch/dataset/keywords/yes/yes.f5626af6_nohash_2.wav"
# _, wav = io.wavfile.read(path)
# wav
# s = base64.b64encode(wav)

# with open ("base64audio", "wb") as f:
#     f.write(s)

# with open("base64audio", "rb") as file:
#     BASE64 = file.read()


# r = base64.decodebytes(BASE64)
# q = np.frombuffer(r, dtype=np.int16)

In [ ]:
from pathlib import Path 
import torchaudio as ta 
import os 
import pandas as pd 
import numpy as numpy 
from sklearn.model_selection import train_test_split

root_dir =Path("/home/akinwilson/Code/pytorch/dataset/keywords") 

# os.listdir(root_dir)

def get_df():
    class_zero = []
    class_zero = [*class_zero, *[str(root_dir / "unknown" / f) for f in os.listdir(root_dir / "unknown")]]
    class_zero = [*class_zero, *[str(root_dir / "no" /  f) for f in os.listdir(root_dir / "no")]]
    class_zero = [*class_zero, *[str(root_dir/ "noise" / f) for f in os.listdir(root_dir / "noise")]]

    class_one = [str(root_dir / "yes" / f) for f in os.listdir(root_dir / "yes")]
    
    class_zero = [f for f in class_zero if f.endswith(".wav")]
    class_one = [f for f in class_one if f.endswith(".wav")]

    records = []
    for p in class_zero:
        records.append({"label": 0.0, "wav_path": p})
    
    for p in class_one:
        records.append({"label": 1.0, "wav_path": p})
    
    return pd.DataFrame.from_records(records)

# 
def create_split(testing=True):
    df = get_df()
    if testing:
        df = df.sample(n=1000, random_state=1)
    dftv, dfs = train_test_split(df, test_size=0.1)
    dft, dfv = train_test_split(dftv, test_size=0.1)
    dft.to_csv(root_dir / "train.csv", index=False)
    dfv.to_csv(root_dir / "val.csv", index=False)
    dfs.to_csv(root_dir / "test.csv", index=False) 

create_split()
# get_df().to_csv(root_dir / "data.csv", index=False)


In [ ]:
import torchaudio as ta 
import torch 

x, sr = ta.load(pd.read_csv("/home/akinwilson/Code/pytorch/dataset/keywords/train.csv").loc[2]['wav_path'])

# x
x =torch.cat([x for _ in range(32)], dim=0)
torch.nn.Linear(16000, 1)(x)

In [ ]:
from torchlibrosa.stft import Spectrogram, LogmelFilterBank
?Spectrogram

In [ ]:
# testing list as arg.parse 
import argparse 
from argparse import ArgumentParser

parser = ArgumentParser 


parser.add_argument('-ex', '--exclude-noise', help='delimited list input', type=str, default="akin,jay")
args = parser.parse_known_args()
my_list = [int(item) for item in args.list.split(',')]

In [ ]:
from datasets import load_dataset
ds = load_dataset('common_voice', 'uk', split='train')
df = pd.DataFrame(ds)



# Sample preparing dataset for train, val and test using dataset lib

In [ ]:
ds1 = load_dataset("csv", data_files=os.path.join(os.environ["SM_CHANNEL_TRAIN"], "train.csv"))
ds2 = load_dataset("csv", data_files=os.path.join(os.environ["SM_CHANNEL_VAL"], "val.csv"))
ds = concatenate_datasets([ds1['train'], ds2['train']])
ds = datasets.DatasetDict({"train":ds})

drop_cols = ['wuw_decision_threshold', 'full_audio_duration', 'device_model',
            'wuw_prediction', 'device_id', 'ttl', 'transaction_id', 'language',
            'full_audio_file_name', 'audio_processing_algorithm', 'device_type',
            'command_primary_transcription', 'sowuw', 'device_sw_version',
            'timestamp', 'return_code', 'device_push_to_talk', 'data_type',
            'wuw_confidence_score', 'hardware_type', 'partner_id', 'eowuw',
            'wuw_model_version', 'wuw_inference_time', 'country_code',
            'annotated_quality', 'annotated_phrase_transcription',
            'annotated_phrase_audio_type', 'annotated_age',
            'annotated_ww_audio_type', 'processed', 'annotated_timestamp',
            'annotated_voice_type', 'annotated_asr_match',
            'annotated_additional_comments', 'wuw_duration', 's3_bucket', 's3_key','response']

ds = ds.remove_columns(drop_cols)

ds = ds.rename_column("wav_path", "audio")
ds = ds.cast_column("audio", Audio())
ClassLabels = ClassLabel(num_classes=len([0.0,1.0]), names=["NOT_WAKEWORD","WAKEWORD"])
ds = ds.cast_column("label", ClassLabels)

ds = ds["train"].train_test_split(
    train_size=0.5, test_size=0.5, stratify_by_column="label"
)

ds["train"] = ds["train"].select(
    [i for i in range((len(ds["train"]) // BATCH_SIZE) * BATCH_SIZE)]
)
ds["test"] = ds["test"].select(
    [i for i in range((len(ds["test"]) // BATCH_SIZE) * BATCH_SIZE)]
)
labels = ds["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

print(id2label)
feature_extractor = AutoFeatureExtractor.from_pretrained(
    MODEL_CHECKPOINT, return_attention_mask=True
)


s = time.time()
logger.info("Starting extraction of features")
ds = ds.map(
    preprocess_function,
    remove_columns=["audio"],
    batched=True)
logger.info("Finished extraction of features")
f = time.time()
logger.info(f"time taken: {f-s}s")

train = ds["train"].shuffle(seed=42).with_format("pt", columns=[ 'label', 'input_values', 'attention_mask'], output_all_columns=True)[:]
test = ds["test"].shuffle(seed=42).with_format("pt", columns=[ 'label', 'input_values', 'attention_mask'], output_all_columns=True)[:]
    